In [ ]:
import importlib
from dasilva_invariants import diagnostics, meshes, invariants
importlib.reload(diagnostics)
importlib.reload(meshes)
importlib.reload(invariants)

In [ ]:
mesh = meshes.get_lfm_hdf4_data('../lfm_data/hdf4/March2015_mhd_2015-03-17T04-45-00Z.hdf')

In [ ]:
diagnostics.tail_traces(mesh, 'LFM')

In [ ]:
diagnostics.tail_K_vs_radius(mesh, 'LFM')

In [ ]:
diagnostics.drift_shells(mesh, 'LFM')

In [ ]:
diagnostics.dayside_field_intensities(mesh, 'LFM')

In [ ]:
diagnostics.equitorial_plot_of_intensity(mesh, 'LFM')

In [ ]:
import numpy as np, pylab as plt
r = 7
th = 180
x, y = np.cos(np.deg2rad(th)) * r, np.sin(np.deg2rad(th)) * r
result = invariants.calculate_K(mesh, (x, y, 0), 7.5)

x, y = np.cos(np.deg2rad(th)) * r, np.sin(np.deg2rad(th)) * r
result = invariants.calculate_K(mesh, (x, y, 0), 7.5)

plt.figure(figsize=(8, 4))

plt.plot(result.integral_axis_latitude, result.integral_integrand, 'k,-')
plt.fill_between(result.integral_axis_latitude,
                 result.integral_integrand.min(),
                 result.integral_integrand)
plt.title(f'K = {result.K:.6f} Re Sqrt(G) = ' '$\int_{s_m}^{s_m\'}\sqrt{B_m - B(s)}ds$\n'
          f'Bm = {result.Bm:.6f} G\n'
          f'Mirror at {result.integral_axis_latitude.max():.1f} deg',
          fontsize=20)
plt.xlabel('Latitude (deg)', fontsize=20)
plt.ylabel('$\sqrt{B_m - B(s)}$', fontsize=20)
None

In [ ]:
import glob
files = glob.glob('../lfm_data/hdf4/*.hdf')[:10]

results = []
for file in files:
    result = None
    mesh = meshes.get_lfm_hdf4_data(file)
    try:
        result = invariants.calculate_LStar(mesh, (4, 0, 0), 7.5, num_local_times=30,
                                        rel_error_threshold=0.03, verbose=True)
        if result:
            results.append((file, result))
    except RuntimeError:
        pass

In [ ]:
plt.figure(figsize=(12, 8))
plt.xlabel('Local time')
plt.ylabel('Radius (Re)')

cmap = plt.get_cmap('viridis')

for i, (file, result) in enumerate(results):
    localtimes = np.rad2deg(result.drift_local_times) * 24 / 360 + 12
    localtimes %= 24
    I = np.argsort(localtimes)
    plt.plot(localtimes[I], result.drift_rvalues[I], '.-', label=file.split('/')[-1], color=cmap(i/len(results)))

plt.legend(ncol=1, loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid()

In [ ]:
plt.figure(figsize=(8, 8))

cmap = plt.get_cmap('viridis')

for i, (file, result) in enumerate(results):
    x = np.cos(result.drift_local_times) * result.drift_rvalues
    y = np.sin(result.drift_local_times) * result.drift_rvalues
    
    x = x.tolist() + [x[0]]
    y = y.tolist() + [y[0]]
    
    plt.plot(x, y, color=cmap(i/len(results)), label=file.split('/')[-1])

plt.legend(ncol=1, loc='center left', bbox_to_anchor=(1, 0.5))
#plt.yscale('symlog')
#plt.xscale('symlog')
plt.grid()